# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "data/schools_complete.csv"
student_data_to_load = "data/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Create a District Summary
school_data_complete.describe()

,Student ID,reading_score,math_score,School ID,size,budget
count,39170.000000,39170.00000,39170.000000,39170.000000,39170.000000,3.917000e+04
mean,19584.500000,81.87784,78.985371,6.978172,3332.957110,2.117241e+06
std,11307.549359,10.23958,12.309968,4.444329,1323.914069,8.749987e+05
min,0.000000,63.00000,55.000000,0.000000,427.000000,2.480870e+05
25%,9792.250000,73.00000,69.000000,3.000000,1858.000000,1.081356e+06
50%,19584.500000,82.00000,79.000000,7.000000,2949.000000,1.910635e+06
75%,29376.750000,91.00000,89.000000,11.000000,4635.000000,3.022020e+06
max,39169.000000,99.00000,99.000000,14.000000,4976.000000,3.124928e+06


In [3]:
# Total number of schools
total_schools = school_data_complete["school_name"].nunique()
print(f'Total number of schools: {total_schools}')

# Total number of students
total_students = len(school_data_complete["student_name"])
print(f'Total number of students: {total_students}')

# Total budget
total_budget = school_data_complete["budget"].sum()
print(f'Total school budget: {total_budget}')

# Average math score
avg_math = school_data_complete["math_score"].mean()
print(f'Average math score: {avg_math}')

# Average reading score
avg_read = school_data_complete["reading_score"].mean()
print(f'Average reading score: {avg_read}')

# Overall average score
overall_score = (avg_math + avg_read)/2
print(f'Overall average score: {overall_score}')

# Percentage of passing math (70 or greater)
pass_math = len(school_data_complete.loc[school_data_complete["math_score"] >= 70])
overall_math = len(school_data_complete)
pass_math_ratio = round(pass_math / overall_math * 100, 2)
print(f'Percentage passing math: {pass_math_ratio}%')

# Percentage of passing math (70 or greater)
pass_read = len(school_data_complete.loc[school_data_complete["reading_score"] >= 70])
overall_read = len(school_data_complete)
pass_read_ratio = round(pass_read / overall_read * 100, 2)
print(f'Percentage passing read: {pass_read_ratio}%')


Total number of schools: 15
Total number of students: 39170
Total school budget: 82932329558
Average math score: 78.98537145774827
Average reading score: 81.87784018381414
Overall average score: 80.43160582078121
Percentage passing math: 74.98%
Percentage passing read: 85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [4]:
# populate school summary table
pass_math = school_data_complete.groupby(['school_name', 'type'])['math_score'].apply(lambda x:(x>70).sum()).reset_index(name='count')


school_summary = school_data_complete.groupby(['school_name', 'type']).agg(
                        total_student = pd.NamedAgg(column='Student ID', aggfunc=len),
                        total_school_budget = pd.NamedAgg(column='budget', aggfunc=sum),
                        avg_math_score = pd.NamedAgg(column='math_score', aggfunc=np.mean),
                        avg_read_score = pd.NamedAgg(column='reading_score', aggfunc=np.mean),
                        pass_math = pd.NamedAgg(column='math_score', aggfunc=lambda x:(x>70).sum()),
                        pass_read = pd.NamedAgg(column='reading_score', aggfunc=lambda x:(x>70).sum())
                        ).reset_index()

school_summary['per_student_budget'] = school_summary['total_school_budget'] / school_summary['total_student']
school_summary['pass_math_ratio'] = round(school_summary['pass_math'] / school_summary['total_student'], 2)
school_summary['pass_read_ratio'] = round(school_summary['pass_read'] / school_summary['total_student'], 2)
school_summary['overall_pass_ratio'] = round((school_summary['pass_math_ratio'] + school_summary['pass_read_ratio'])/2, 2)
school_summary

# sort and display the top five schools in overall passing rate
school_summary.sort_values(['overall_pass_ratio', 
                            'pass_math_ratio', 
                            'pass_read_ratio'], ascending=False).head(5)



,school_name,type,total_student,total_school_budget,avg_math_score,avg_read_score,pass_math,pass_read,per_student_budget,pass_math_ratio,pass_read_ratio,overall_pass_ratio
9,Pena High School,Charter,962,563595396,83.839917,84.044699,882,887,585858.0,0.92,0.92,0.92
6,Holden High School,Charter,427,105933149,83.803279,83.814988,387,396,248087.0,0.91,0.93,0.92
13,Wilson High School,Charter,2283,3012587442,83.274201,83.989488,2076,2129,1319574.0,0.91,0.93,0.92
1,Cabrera High School,Charter,1858,2009159448,83.061895,83.975780,1664,1744,1081356.0,0.90,0.94,0.92
4,Griffin High School,Charter,1468,1346890000,83.351499,83.816757,1317,1371,917500.0,0.90,0.93,0.92


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [5]:
# bottom performing schools
school_summary.sort_values(['overall_pass_ratio', 
                            'pass_math_ratio', 
                            'pass_read_ratio']).head(5)

,school_name,type,total_student,total_school_budget,avg_math_score,avg_read_score,pass_math,pass_read,per_student_budget,pass_math_ratio,pass_read_ratio,overall_pass_ratio
7,Huang High School,District,2917,5573322295,76.629414,81.182722,1847,2299,1910635.0,0.63,0.79,0.71
2,Figueroa High School,District,2949,5557128039,76.711767,81.158020,1880,2313,1884411.0,0.64,0.78,0.71
8,Johnson High School,District,4761,14733628650,77.072464,80.966394,3040,3727,3094650.0,0.64,0.78,0.71
10,Rodriguez High School,District,3999,10186904637,76.842711,80.744686,2562,3109,2547363.0,0.64,0.78,0.71
5,Hernandez High School,District,4635,14007062700,77.289752,80.934412,3001,3624,3022020.0,0.65,0.78,0.72


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [6]:
# Create table that lists the average math score for each school of each grade level.

# Calculate the average math score for 9th grade in each school
grade_9_df = school_data_complete.loc[school_data_complete["grade"] == "9th", :]
grade_9_math = grade_9_df.groupby(["school_name"])['math_score'].mean()


# Calculate the average math score for 10th grade in each school
grade_10_df = school_data_complete.loc[school_data_complete["grade"] == "10th", :]
grade_10_math = grade_10_df.groupby(["school_name"])['math_score'].mean()


# Calculate the average math score for 11th grade in each school
grade_11_df = school_data_complete.loc[school_data_complete["grade"] == "11th", :]
grade_11_math = grade_11_df.groupby(["school_name"])['math_score'].mean()


# Calculate the average math score for 12th grade in each school
grade_12_df = school_data_complete.loc[school_data_complete["grade"] == "12th", :]
grade_12_math = grade_12_df.groupby(["school_name"])['math_score'].mean()



pd.DataFrame({'9th':grade_9_math,
             '10th':grade_10_math,
             '11th':grade_11_math,
             '12th':grade_12_math}).reset_index()

,school_name,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [7]:
# or using pivot table
pd.pivot_table(school_data_complete, 
               index='school_name',
               columns='grade',
               values='math_score',
               aggfunc=np.mean
              )

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010
Hernandez High School,77.337408,77.136029,77.186567,77.438495
Holden High School,83.429825,85.000000,82.855422,83.787402
Huang High School,75.908735,76.446602,77.225641,77.027251
Johnson High School,76.691117,77.491653,76.863248,77.187857


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
# Create table that lists the average reading score for each school of each grade level.

# Calculate the average reading score for 9th grade in each school
grade_9_read = grade_9_df.groupby(["school_name"])['reading_score'].mean()

# Calculate the average reading score for 10th grade in each school
grade_10_read = grade_10_df.groupby(["school_name"])['reading_score'].mean()

# Calculate the average reading score for 11th grade in each school
grade_11_read = grade_11_df.groupby(["school_name"])['reading_score'].mean()

# Calculate the average reading score for 12th grade in each school
grade_12_read = grade_12_df.groupby(["school_name"])['reading_score'].mean()

pd.DataFrame({'9th':grade_9_read,
             '10th':grade_10_read,
             '11th':grade_11_read,
             '12th':grade_12_read}).reset_index()

,school_name,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 600, 625, 650, 1000]
group_names = ["<$585", "$585-600", "$600-625", "$625-650", "$650<"]

In [10]:
# Create a new column to show budget per student in each row
school_data_complete["spending per student"] = school_data_complete["budget"] / school_data_complete["size"]

# Create a new column to define the spending ranges per student
school_data_complete["spending"] = pd.cut(
    school_data_complete["spending per student"], spending_bins, labels=group_names)

# Calculate the average math score within each spending range
spending_range_df = school_data_complete.groupby(["spending"])
avg_math_range = spending_range_df["math_score"].mean().reset_index()
avg_read_range = spending_range_df["reading_score"].mean().reset_index()

# join the math and reading tables
avg_score = avg_math_range.merge(avg_read_range)

# Calculate the percentage passing rate for math in each spending range
spending_math_pass = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
spending_math_pass_df = spending_math_pass.groupby(["spending"])
math_pass_sp = spending_math_pass_df["math_score"].count()
math_total_sp = spending_range_df["math_score"].count()
math_percent_pass = math_pass_sp / math_total_sp
math_percent_pass = math_percent_pass.reset_index()

# Calculate the percentage passing rate for reading in each spending range
spending_read_pass = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
spending_read_pass_df = spending_read_pass.groupby(["spending"])
read_pass_sp = spending_read_pass_df["reading_score"].count()
read_total_sp = spending_range_df["reading_score"].count()
read_percent_pass = read_pass_sp / read_total_sp
read_percent_pass = read_percent_pass.reset_index()

# join the math and read tables
percent_pass = math_percent_pass.merge(read_percent_pass)
percent_pass = percent_pass.rename({'math_score':'math_pass',
                                   'reading_score':'read_pass'},
                                  axis=1)

school_spending_df = percent_pass.merge(avg_score, how='inner',
                                       left_on='spending',
                                       right_on='spending')

# Calculate the percentage overall passing rate in each spending range
school_spending_df['overall_score'] = (school_spending_df['math_pass'] + school_spending_df['read_pass']) / 2

school_spending_df

,spending,math_pass,read_pass,math_score,reading_score,overall_score
0,<$585,0.937029,0.966866,83.363065,83.964039,0.951947
1,$585-600,0.938671,0.958546,83.359455,83.725724,0.948609
2,$600-625,0.938683,0.966667,83.544856,83.906996,0.952675
3,$625-650,0.686595,0.821312,77.469253,81.162258,0.753953
4,$650<,0.663400,0.810381,77.034693,81.030323,0.736891


## Scores by School Size

* Perform the same operations as above, based on school size.

In [11]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [12]:
# Create a new column for the bin groups
school_data_complete["School Size"] = pd.cut(school_data_complete["size"],
                                             size_bins, labels=group_names)
# school_data_complete
size_df = school_data_complete.groupby(["School Size"])
avg_math_size = size_df["math_score"].mean()
avg_read_read = size_df["reading_score"].mean()
test_count_size = size_df["Student ID"].count()

# Look for the total count of test scores that pass 70% or higher
math_pass_size = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
# math_pass_size
math_pass_size_df = math_pass_size.groupby(["School Size"])
math_count_size = math_pass_size_df["math_score"].count()
math_pass_rate_size = math_count_size / test_count_size


read_pass_size = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
# read_pass_size
read_pass_size_df = read_pass_size.groupby(["School Size"])
read_count_size = read_pass_size_df["reading_score"].count()
read_pass_rate_size = read_count_size / test_count_size

# Calculate the overall passing rate for different school size
overall_pass_rate_size = (math_pass_rate_size + read_pass_rate_size) / 2

school_size_df = pd.DataFrame({'math_pass':math_pass_rate_size ,
                               'read_pass':read_pass_rate_size, 
                               'overall_pass':overall_pass_rate_size}).reset_index()
school_size_df


,School Size,math_pass,read_pass,overall_pass
0,Small (<1000),0.939525,0.960403,0.949964
1,Medium (1000-2000),0.936165,0.967731,0.951948
2,Large (2000-5000),0.686524,0.821252,0.753888


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
# Create bins and groups
school_type = school_data_complete.groupby(["type"])
avg_math_type = school_type["math_score"].mean()
avg_read_type = school_type["reading_score"].mean()
test_count_type = school_type["Student ID"].count()

# Find counts of the passing 70 or higher score for the both test
math_pass_type = school_data_complete.loc[school_data_complete["math_score"] >= 70, :]
math_pass_type_df = math_pass_type.groupby(["type"])
math_count_type = math_pass_type_df["math_score"].count()
math_pass_rate_type = math_count_type / test_count_type


read_pass_type = school_data_complete.loc[school_data_complete["reading_score"] >= 70, :]
read_pass_type_df = read_pass_type.groupby(["type"])
read_count_type = read_pass_type_df["reading_score"].count()
read_pass_rate_type = read_count_type / test_count_type


# Calculate the overall passing rate
overall_pass_rate_type = (math_pass_rate_type + read_pass_rate_type) / 2

pd.DataFrame({'math_pass':math_pass_rate_type,
              'read_pass':read_pass_rate_type,
              'overall_pass':overall_pass_rate_type})



,math_pass,read_pass,overall_pass
type,,,
Charter,0.937018,0.966459,0.951739
District,0.665184,0.809052,0.737118
